# Drive Mounting and Dependencies



In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
%cd /gdrive/MyDrive/SamsungWork/AffectiveROAD_Dataset/Database/E4/1-E4-Drv1/Left

/gdrive/MyDrive/SamsungWork/AffectiveROAD_Dataset/Database/E4/1-E4-Drv1/Left


In [ ]:
!pip install heartpy
!pip install pyhrv
!pip install biosppy
!pip install pyhrv
!pip install hrv-analysis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 4.3 MB/s 
     |████████████████████████████████| 1.5 MB 44.6 MB/s 
     |████████████████████████████████| 230 kB 45.0 MB/s 
     |████████████████████████████████| 47 kB 3.3 MB/s 
  Created wheel for biosppy: filename=biosppy-0.8.0-py2.py3-none-any.whl size=100176 sha256=d9d3d9ef27db8c18b9356bc8719f5d0b256f7e1b8ea4fabbef130edc65b5beaa
  Stored in directory: /root/.cache/pip/wheels/32/17/9d/77adbf0db713546eaa1c7a1480b2dce91fa724a133115dc20c
  Created wheel for spectrum: filename=spectrum-0.8.1-cp37-cp37m-linux_x86_64.whl size=235154 sha256=e593f04adaede18b7655d1f1791888d9829e0421e6e856f55fee39b495981cce
  Stored in directory: /root/.cache/pip/wheels/79/db/9c/92fa684ca088447807d08672e7609b

In [ ]:
import os
import sys
import csv


import numpy as np
import sympy as sp
import weakref
import matplotlib.pyplot as plt
import math
import statistics


#median
import statistics
from statistics import median

import os
from glob import glob

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz
from scipy.signal import butter, filtfilt, tf2zpk


import heartpy as hp

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth',100000)
import cvxopt as cv
import cvxopt.solvers

import scipy.stats as sp

from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
from hrvanalysis import *

# Feature Extraction Functions- Setup

## GSR

### GSR Helper Functions

#### Detect Peak, cvxEDA and feature extraction function

In [ ]:

def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None):

    """Detect peaks in data based on their amplitude and other features.
    Parameters
    ----------
    x : 1D array_like
        data.
    mph : {None, number}, optional (default = None)
        detect peaks that are greater than minimum peak height.
    mpd : positive integer, optional (default = 1)
        detect peaks that are at least separated by minimum peak distance (in
        number of data).
    threshold : positive number, optional (default = 0)
        detect peaks (valleys) that are greater (smaller) than `threshold`
        in relation to their immediate neighbors.
    edge : {None, 'rising', 'falling', 'both'}, optional (default = 'rising')
        for a flat peak, keep only the rising edge ('rising'), only the
        falling edge ('falling'), both edges ('both'), or don't detect a
        flat peak (None).
    kpsh : bool, optional (default = False)
        keep peaks with same height even if they are closer than `mpd`.
    valley : bool, optional (default = False)
        if True (1), detect valleys (local minima) instead of peaks.
    show : bool, optional (default = False)
        if True (1), plot data in matplotlib figure.
    ax : a matplotlib.axes.Axes instance, optional (default = None).
    Returns
    -------
    ind : 1D array_like
        indeces of the peaks in `x`.
    Notes
    -----
    The detection of valleys instead of peaks is performed internally by simply
    negating the data: `ind_valleys = detect_peaks(-x)`
    
    The function can handle NaN's 
    See this IPython Notebook [1]_.
    References
    ----------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
    Examples
    --------
    >>> from detect_peaks import detect_peaks
    >>> x = np.random.randn(100)
    >>> x[60:81] = np.nan
    >>> # detect all peaks and plot data
    >>> ind = detect_peaks(x, show=True)
    >>> print(ind)
    >>> x = np.sin(2*np.pi*5*np.linspace(0, 1, 200)) + np.random.randn(200)/5
    >>> # set minimum peak height = 0 and minimum peak distance = 20
    >>> detect_peaks(x, mph=0, mpd=20, show=True)
    >>> x = [0, 1, 0, 2, 0, 3, 0, 2, 0, 1, 0]
    >>> # set minimum peak distance = 2
    >>> detect_peaks(x, mpd=2, show=True)
    >>> x = np.sin(2*np.pi*5*np.linspace(0, 1, 200)) + np.random.randn(200)/5
    >>> # detection of valleys instead of peaks
    >>> detect_peaks(x, mph=0, mpd=20, valley=True, show=True)
    >>> x = [0, 1, 1, 0, 1, 1, 0]
    >>> # detect both edges
    >>> detect_peaks(x, edge='both', show=True)
    >>> x = [-2, 1, -2, 2, 1, 1, 3, 0]
    >>> # set threshold = 2
    >>> detect_peaks(x, threshold = 2, show=True)
    """

    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                    & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])

    if show:
        if indnan.size:
            x[indnan] = np.nan
        if valley:
            x = -x
        _plot(x, mph, mpd, threshold, edge, valley, ax, ind)

    return ind


In [ ]:
def movingAvg(dt):
    N = 16
    cumsum, moving_aves = [0], []
    for i, x in enumerate(dt, 1):
        cumsum.append(cumsum[i-1] + x)
        if i>=N:
            moving_ave = (cumsum[i] - cumsum[i-N])/N
            moving_aves.append(moving_ave)
    return moving_aves

In [ ]:
def cvxEDA(y, delta, tau0=2., tau1=0.7, delta_knot=10., alpha=8e-4, gamma=1e-2,
           solver=None, options={'reltol':1e-9}):
    """CVXEDA Convex optimization approach to electrodermal activity processing
    This function implements the cvxEDA algorithm described in "cvxEDA: a
    Convex Optimization Approach to Electrodermal Activity Processing"
    (http://dx.doi.org/10.1109/TBME.2015.2474131, also available from the
    authors' homepages).
    Arguments:
       y: observed EDA signal (we recommend normalizing it: y = zscore(y))
       delta: sampling interval (in seconds) of y
       tau0: slow time constant of the Bateman function
       tau1: fast time constant of the Bateman function
       delta_knot: time between knots of the tonic spline function
       alpha: penalization for the sparse SMNA driver
       gamma: penalization for the tonic spline coefficients
       solver: sparse QP solver to be used, see cvxopt.solvers.qp
       options: solver options, see:
                http://cvxopt.org/userguide/coneprog.html#algorithm-parameters
    Returns (see paper for details):
       r: phasic component
       p: sparse SMNA driver of phasic component
       t: tonic component
       l: coefficients of tonic spline
       d: offset and slope of the linear drift term
       e: model residuals
       obj: value of objective function being minimized (eq 15 of paper)
    """

    n = len(y)
    y = cv.matrix(y)

    # bateman ARMA model
    a1 = 1./min(tau1, tau0) # a1 > a0
    a0 = 1./max(tau1, tau0)
    ar = np.array([(a1*delta + 2.) * (a0*delta + 2.), 2.*a1*a0*delta**2 - 8.,
        (a1*delta - 2.) * (a0*delta - 2.)]) / ((a1 - a0) * delta**2)
    ma = np.array([1., 2., 1.])

    # matrices for ARMA model
    i = np.arange(2, n)
    A = cv.spmatrix(np.tile(ar, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))
    M = cv.spmatrix(np.tile(ma, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))

    # spline
    delta_knot_s = int(round(delta_knot / delta))
    spl = np.r_[np.arange(1.,delta_knot_s), np.arange(delta_knot_s, 0., -1.)] # order 1
    spl = np.convolve(spl, spl, 'full')
    spl /= max(spl)
    # matrix of spline regressors
    i = np.c_[np.arange(-(len(spl)//2), (len(spl)+1)//2)] + np.r_[np.arange(0, n, delta_knot_s)]
    nB = i.shape[1]
    j = np.tile(np.arange(nB), (len(spl),1))
    p = np.tile(spl, (nB,1)).T
    valid = (i >= 0) & (i < n)
    B = cv.spmatrix(p[valid], i[valid], j[valid])

    # trend
    C = cv.matrix(np.c_[np.ones(n), np.arange(1., n+1.)/n])
    nC = C.size[1]

    # Solve the problem:
    # .5*(M*q + B*l + C*d - y)^2 + alpha*sum(A,1)*p + .5*gamma*l'*l
    # s.t. A*q >= 0

    old_options = cv.solvers.options.copy()
    cv.solvers.options.clear()
    cv.solvers.options.update(options)
    if solver == 'conelp':
        # Use conelp
        z = lambda m,n: cv.spmatrix([],[],[],(m,n))
        G = cv.sparse([[-A,z(2,n),M,z(nB+2,n)],[z(n+2,nC),C,z(nB+2,nC)],
                    [z(n,1),-1,1,z(n+nB+2,1)],[z(2*n+2,1),-1,1,z(nB,1)],
                    [z(n+2,nB),B,z(2,nB),cv.spmatrix(1.0, range(nB), range(nB))]])
        h = cv.matrix([z(n,1),.5,.5,y,.5,.5,z(nB,1)])
        c = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T,z(nC,1),1,gamma,z(nB,1)])
        res = cv.solvers.conelp(c, G, h, dims={'l':n,'q':[n+2,nB+2],'s':[]})
        obj = res['primal objective']
    else:
        # Use qp
        Mt, Ct, Bt = M.T, C.T, B.T
        H = cv.sparse([[Mt*M, Ct*M, Bt*M], [Mt*C, Ct*C, Bt*C], 
                    [Mt*B, Ct*B, Bt*B+gamma*cv.spmatrix(1.0, range(nB), range(nB))]])
        f = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T - Mt*y,  -(Ct*y), -(Bt*y)])
        res = cv.solvers.qp(H, f, cv.spmatrix(-A.V, A.I, A.J, (n,len(f))),
                            cv.matrix(0., (n,1)), solver=solver)
        obj = res['primal objective'] + .5 * (y.T * y)
    cv.solvers.options.clear()
    cv.solvers.options.update(old_options)

    l = res['x'][-nB:]
    d = res['x'][n:n+nC]
    t = B*l + C*d
    q = res['x'][:n]
    p = A * q
    r = M * q
    e = y - r - t

    return (np.array(a).ravel() for a in (r, p, t, l, d, e, obj))




In [ ]:
def process(y,delta,state): # if state=0, then rest, 1- mid, 2- high
    [r, p, t, l, d, e, obj] = cvxEDA(y, delta)
    smoothed_phasic=movingAvg(r)
    phasic_peaks=detect_peaks(smoothed_phasic, mph=0.001, mpd=int(1/delta), show=False)
    temp={}
    #Calculate the moments of the GSR data
    temp["mean_gsr"]=np.mean(y)
    temp["var_gsr"]=np.var(y)
    temp["skew_gsr"]=sp.skew(y)
    temp["kurtosis_gsr"]=sp.kurtosis(y)
    temp["std_gsr"]=np.std(y)
    #Calculate the moments of the SCL data (tonic)
    temp["mean_scl"]=np.mean(t)
    temp["var_scl"]=np.var(t)
    temp["skew_scl"]=sp.skew(t)
    temp["kurtosis_scl"]=sp.kurtosis(t)
    temp["std_scl"]=np.std(t)
    temp["slope_scl"]=np.max(t)-np.min(t)
    #Calculate the moments of the SCR data (phasic)
    temp["mean_scr"]=np.mean(r)
    temp["var_scr"]=np.var(r)
    temp["skew_scr"]=sp.skew(r)
    temp["kurtosis_scr"]=sp.kurtosis(r)
    temp["std_scr"]=np.std(r)
    temp["max_scr"]=np.max(r)
    temp["scr_peaks"]=len(phasic_peaks)
    temp["stress"]=state
    return temp

### GSR Window & Main Code

In [ ]:
def gsrwindowing(df,windowsizerow,state):
  feature=['mean_gsr','var_gsr','skew_gsr','kurtosis_gsr','std_gsr','mean_scl','var_scl','skew_scl','kurtosis_scl','std_scl','slope_scl','mean_scr','var_scr','skew_scr','kurtosis_scr','std_scr','max_scr','scr_peaks','stress']
  result_df= pd.DataFrame(columns=feature) 
  df1=df.rolling(window=windowsizerow)
  type(df1)
  i=1
  j=0
  for df2 in df1:
    if len(df2)>=windowsizerow and i%int(windowsizerow/2)==0:
     results=process(df2.GSR,1./64,state)
     result_df=result_df.append(results,ignore_index=True)
   
    i+=1
    j+=1
    
  return result_df

In [ ]:
def GSR_FeatureExtraction(drive, index):
  annot=pd.read_csv(r'/gdrive/MyDrive/SamsungWork/AffectiveROAD_Dataset/Database/E4/Annot_E4_Left.csv')
  gsr=pd.read_csv(os.path.join(r'/gdrive/MyDrive/SamsungWork/AffectiveROAD_Dataset/Database/ExtractedE4Files/E4',drive,"Left","EDA.csv"),names=['GSR'])
  gsr=gsr[2:]
  gsr.reset_index(drop=True,inplace=True)
  mul_factor=64//4
  upsampled_gsr=signal.resample(gsr,len(gsr)*mul_factor)
  gsr = pd.DataFrame(upsampled_gsr, columns =['GSR'])

  gsrfeaturesname=['mean_gsr','var_gsr','skew_gsr','kurtosis_gsr','std_gsr','mean_scl','var_scl','skew_scl','kurtosis_scl','std_scl','slope_scl','mean_scr','var_scr','skew_scr','kurtosis_scr','std_scr','max_scr','scr_peaks','stress']
  gsrresultdf= pd.DataFrame(columns=gsrfeaturesname) 



  # rest features computation 
  rest=gsr[annot['Rest_Start'][index]*mul_factor:annot['Rest_End'][index]*mul_factor]
  rest=rest.append(gsr[annot['Rest_Start.1'][index]*mul_factor:annot['Rest_End.1'][index]*mul_factor],ignore_index=True)
  rest.reset_index(drop=True,inplace=True)
  restfeatures=gsrwindowing(rest,64*10,0)
  gsrresultdf=gsrresultdf.append(restfeatures,ignore_index=True)

    # medium stress features computation-> Highway
  medium=gsr[annot['Hwy_Start'][index]*mul_factor:annot['Hwy_End'][index]*mul_factor]
  medium=medium.append(gsr[annot['Hwy_Start.1'][index]*mul_factor:annot['Hwy_End.1'][index]*mul_factor],ignore_index=True)
  medium.reset_index(drop=True,inplace=True)
  mediumfeatures=gsrwindowing(medium,64*10,1)
  gsrresultdf=gsrresultdf.append(mediumfeatures,ignore_index=True)

    # high stress features computation-> Highway
  high=gsr[annot['City1_Start'][index]*mul_factor:annot['City1_End'][index]*mul_factor]
  high=high.append(gsr[annot['City1_Start.1'][index]*mul_factor:annot['City1_End.1'][index]*mul_factor],ignore_index=True)
  high=gsr[annot['City2_Start'][index]*mul_factor:annot['City2_End'][index]*mul_factor]
  high=high.append(gsr[annot['City2_Start.1'][index]*mul_factor:annot['City2_End.1'][index]*mul_factor],ignore_index=True)
  high.reset_index(drop=True,inplace=True)
  highfeatures=gsrwindowing(high,64*10,2)
  gsrresultdf=gsrresultdf.append(highfeatures,ignore_index=True)

  return gsrresultdf



## BVP

### BVP Helper Functions

In [ ]:
def bandpass_butterworth(filter_order, cutoff_frequency_low, cutoff_frequency_high, sampling_frequency):
    """Creating a Butterworth bandpass filter, with a frequency band from cutoff_frequency_low - cutoff_frequency_high.
    """
    nyquist_frequency = 0.5 * sampling_frequency
    low_cut = cutoff_frequency_low / nyquist_frequency
    high_cut = cutoff_frequency_high / nyquist_frequency
    b, a = butter(filter_order, [low_cut, high_cut], btype='band')
    print(type(a))
    print(type(b))
    return b, a


def zero_phase_filtering(data, f_min, f_max, sampling_frequency, filter_order=2):
    """Filter function using a zero phase second order Butterworth filter, with bandpass f_min - f_max, is implemented
    to remove baseline wander and high frequencies. Returns zero phase bandpass filtered signal s_n"""
    # test if data is a numpy array
    if type(data) is not np.ndarray:
        signal = np.array(data)
    else:
        signal = data

    # create a bandpass butterworth filter
    b, a = bandpass_butterworth(filter_order=filter_order, cutoff_frequency_low=f_min, cutoff_frequency_high=f_max,
                                sampling_frequency=sampling_frequency)

    # We use scypi.filtfilt using the Gustafsson's method (include paper)
    # The irlen argument can be used to improve the performance of Gustafsson’s method.
    # (url: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.filtfilt.html#r10879a509a76-1)
    # Estimate the impulse response length of the filter.
    z, p, k = tf2zpk(b, a)
    eps = 1e-9
    r = np.max(np.abs(p))
    approx_impulse_len = int(np.ceil(np.log(eps) / np.log(r)))
    # apply the zero phase bandpass filter with estimated impuls length according to Gustafsson's method
    s_n_gust = filtfilt(b, a, signal, method='gust', irlen=approx_impulse_len)
   
    return s_n_gust


def clipping(data):
    """Clipping the input signal by keeping the signal above 0. Function will produce the output signal z_n."""
    # test if data is a numpy array
    if type(data) is not np.ndarray:
        s_n = np.array(data)
    else:
        s_n = data

    z_n = np.zeros(len(s_n))

    for e in range(len(s_n)):
        if s_n[e] >= 0:
            np.put(z_n, [e], s_n[e])
        else:
            np.put(z_n, [e], 0)
    # # plot the result
    # plt.figure(dpi=1200)
    # plt.plot(s_n, linewidth=1.0, label='input')
    # plt.plot(z_n, linewidth=1.0, label='clipped')
    # plt.show()

    return z_n


def squaring(data):
    """Squaring the input signal to emphasize large differences resulting from systolic waves,
    while suppressing small differences arising from diastolic waves and noise. Function produces output y_n"""
    # test if data is a numpy array
    if type(data) is not np.ndarray:
        z_n = np.array(data)
    else:
        z_n = data

    y_n = np.square(z_n)
 

    return y_n


def generate_moving_averages(data, sampling_frequency, window_for_peak, window_for_beat):
    if type(data) is not np.ndarray:
        y_n = np.array(data)
    else:
        y_n = data

    # the window is calculated from the desired time window (window_for_peak) and the sampling frequency
    # this is done to get the number of samples N that make up the window
    N_1 = math.floor(window_for_peak * sampling_frequency)
    # next we have to guarantee the window extends equally to both sides of the current sample
    # therefore we provide it is the size of the nearest uneven integer
    if N_1 % 2 == 0:
        N_1 = N_1 + 1

    # we calculate the moving avg by convolving with a kernel function
    # the kernel has the size of our window and his coefficients provide the same result as a mov avg
    # coeff = 1/N, N being the window size
    # modes are valid, full and same
    moving_average_peak = np.convolve(y_n, np.ones((N_1,)) / N_1, mode='same')
    # repeat for beat region of interest and beat mov avg
    N_2 = math.floor(window_for_beat * sampling_frequency)
    if N_2 % 2 == 0:
        N_2 = N_2 + 1

    moving_average_beat = np.convolve(y_n, np.ones((N_2,)) / N_2, mode='same')
  

    return moving_average_peak, moving_average_beat, N_1


def thresholding(data, moving_average_beat, window_for_peak_in_samples):
    beta = 0.02
    z_mean = statistics.mean(data)
    offset_alpha = beta * z_mean

    first_dynamic_threshold = moving_average_beat + offset_alpha
    second_static_threshold = window_for_peak_in_samples

    return first_dynamic_threshold, second_static_threshold


def find_valid_peaks(block_array, signal, second_threshold):
    diff_mask = np.diff(block_array, n=1,)


    get_flanks = np.argwhere(diff_mask != 0)

    if diff_mask[get_flanks[0]] < diff_mask[get_flanks[1]]:
        start_index = 1
    else:
        start_index = 0

    get_flanks = get_flanks[start_index:]
    valid_peaks_amplitude = np.zeros((len(get_flanks)))
    valid_peaks_location = np.zeros((len(get_flanks)))

    for f in range(0, len(get_flanks)-1, 2):
        block_start = get_flanks[f][0]
        block_end = get_flanks[f+1][0]
        block_width = block_end - block_start

        # signal_block = signal[block_start:block_end]
        signal_block = signal[get_flanks[f][0]:get_flanks[f + 1][0]]

        if block_width >= second_threshold:
            block_maximum = np.amax(signal_block)
            index_maximum = np.argwhere(signal_block == block_maximum)

            np.put(valid_peaks_amplitude, [f][0], block_maximum)
            np.put(valid_peaks_location, [f][0], index_maximum + block_start)

    valid_peaks_amplitude = valid_peaks_amplitude[valid_peaks_amplitude > 0]
    valid_peaks_location = valid_peaks_location[valid_peaks_location > 0]

    return valid_peaks_amplitude, valid_peaks_location



def peak_detection_bvp(raw_data):
    fs = 64
    # step 1: Bandpass filtering
    filtered_signal = zero_phase_filtering(data=raw_data, f_min=0.5, f_max=8, sampling_frequency=fs, filter_order=2)
    # step 2: Clipping
    clipped_signal = clipping(filtered_signal)
    # step 3: Squaring
    squared_clipped_signal = squaring(clipped_signal)
    # step 4: Moving averages
    MA_peak, MA_beat, samples_peak = generate_moving_averages(data=squared_clipped_signal,
                                                              sampling_frequency=fs, window_for_peak=0.111
                                                              , window_for_beat=0.667)
    # step 5: Thresholds and Offset
    first_threshold, second_threshold = thresholding(data=squared_clipped_signal, moving_average_beat=MA_beat,
                                                     window_for_peak_in_samples=samples_peak)
    # step 6: generate blocks of interest
    blocks_of_interest = np.zeros(len(MA_peak))

    for n in range(len(MA_peak)):
        if MA_peak[n] >= first_threshold[n]:
            np.put(blocks_of_interest, [n], 5000.0)
        else:
            np.put(blocks_of_interest, [n], 0.0)

   

    # step 7: find valid blocks
    peak_amplitudes, peak_locations = find_valid_peaks(blocks_of_interest, squared_clipped_signal, second_threshold)

    rr = np.diff(peak_locations)

    return rr

### BVP Window & Main Code

In [ ]:
def bvpwindowing(df,windowsizerow,state):
  feature=['mean_nni','sdnn','sdsd','rmssd','median_nni','nni_50','pnni_50','nni_20','pnni_20','range_nni','cvsd','cvnni','mean_hr','max_hr','min_hr','std_hr','triangular_index','tinn','total_power','vlf','lf','hf','lf_hf_ratio','lfnu','hfnu','csi','cvi','Modified_csi','sd1','sd2','ratio_sd2_sd1','sampen','stress']
  result_df= pd.DataFrame(columns=feature) 
  df1=df.rolling(window=windowsizerow)
  type(df1)
  i=1
  j=0
  for df2 in df1:
    if len(df2)>=windowsizerow and i%int(windowsizerow/2)==0:
      results_td=[]
      results_gd=[]
      results_fd=[]
      results_ccd=[]
      results_ppd=[]
      results_sampen=[]
      results=[]
      rr_corrected= peak_detection_bvp(df2.values.flatten())
  

      interpolated_rr_intervals = interpolate_nan_values(rr_corrected,interpolation_method="linear")

      results_td=get_time_domain_features(interpolated_rr_intervals)
      results_gd=get_geometrical_features(interpolated_rr_intervals)
      results_fd=get_frequency_domain_features(interpolated_rr_intervals,"welch",256)
      results_ccd=get_csi_cvi_features(interpolated_rr_intervals)
      results_ppd=get_poincare_plot_features(interpolated_rr_intervals)
      results_sampen=get_sampen(interpolated_rr_intervals)


      results={**results_td,**results_gd,**results_fd,**results_ccd,**results_ppd,**results_sampen,**{'stress':state}}
      
      result_df=result_df.append(results,ignore_index=True)
   
    i+=1
    j+=1
    
  return result_df

In [ ]:
def BVP_FeatureExtraction(drive, index):
  annot=pd.read_csv(r'/gdrive/MyDrive/SamsungWork/AffectiveROAD_Dataset/Database/E4/Annot_E4_Left.csv')
  gsr=pd.read_csv(os.path.join(r'/gdrive/MyDrive/SamsungWork/AffectiveROAD_Dataset/Database/ExtractedE4Files/E4',drive,"Left","BVP.csv"),names=['BVP'])
  bvp=gsr[2:]
  bvp.reset_index(drop=True,inplace=True)
  mul_factor=64//4

  bvpfeaturesname=['mean_nni','sdnn','sdsd','rmssd','median_nni','nni_50','pnni_50','nni_20','pnni_20','range_nni','cvsd','cvnni','mean_hr','max_hr','min_hr','std_hr','triangular_index','tinn','total_power','vlf','lf','hf','lf_hf_ratio','lfnu','hfnu','csi','cvi','Modified_csi','sd1','sd2','ratio_sd2_sd1','sampen','stress']
  bvpresultdf= pd.DataFrame(columns=bvpfeaturesname) 



  # rest features computation 
  rest=bvp[annot['Rest_Start'][index]*mul_factor:annot['Rest_End'][index]*mul_factor]
  rest=rest.append(bvp[annot['Rest_Start.1'][index]*mul_factor:annot['Rest_End.1'][index]*mul_factor],ignore_index=True)
  rest.reset_index(drop=True,inplace=True)
  restfeatures=bvpwindowing(rest,64*10,0)
  bvpresultdf=bvpresultdf.append(restfeatures,ignore_index=True)

    # medium stress features computation-> Highway
  medium=bvp[annot['Hwy_Start'][index]*mul_factor:annot['Hwy_End'][index]*mul_factor]
  medium=medium.append(bvp[annot['Hwy_Start.1'][index]*mul_factor:annot['Hwy_End.1'][index]*mul_factor],ignore_index=True)
  medium.reset_index(drop=True,inplace=True)
  mediumfeatures=bvpwindowing(medium,64*10,1)
  bvpresultdf=bvpresultdf.append(mediumfeatures,ignore_index=True)

    # high stress features computation-> Highway
  high=bvp[annot['City1_Start'][index]*mul_factor:annot['City1_End'][index]*mul_factor]
  high=high.append(bvp[annot['City1_Start.1'][index]*mul_factor:annot['City1_End.1'][index]*mul_factor],ignore_index=True)
  high=bvp[annot['City2_Start'][index]*mul_factor:annot['City2_End'][index]*mul_factor]
  high=high.append(bvp[annot['City2_Start.1'][index]*mul_factor:annot['City2_End.1'][index]*mul_factor],ignore_index=True)
  high.reset_index(drop=True,inplace=True)
  highfeatures=bvpwindowing(high,64*10,2)
  bvpresultdf=bvpresultdf.append(highfeatures,ignore_index=True)

  return bvpresultdf



## Skin Temperature Features

### Skin Temperature Helper functions

In [ ]:
def _generate_moving_averages(data: np.ndarray, sampling_frequency: int, window: float, mode: str,
                                 verbose: bool = False) -> np.ndarray:
    """
    Function that generates a moving averages.
    Parameters
    ----------
    data: np.ndarray
        one dimensional input signal.
    sampling_frequency: int
        sampling frequency used in acquisition of data parameter.
    window: float
        sets window size for peak detection.
    mode: str
        convolution mode, decides how signal edges are handled.
    verbose: bool
        print information about function progress.
    Returns
    ----------
    moving_average: np.ndarray
        mov avg.
    References
    -----------
    No references needed.
    """
    if type(data) is not np.ndarray:
        y_n = np.array(data)
    else:
        y_n = data

    # the window is calculated from the desired time window (window_for_peak) and the sampling frequency
    # this is done to get the number of samples N that make up the window
    N_1 = math.floor(window * sampling_frequency)
    # next we have to guarantee the window extends equally to both sides of the current sample
    # therefore we provide it is the size of the nearest uneven integer
    if N_1 % 2 == 0:
        N_1 = N_1 + 1

    # we calculate the moving avg by convolving with a kernel function
    # the kernel has the size of our window and his coefficients provide the same result as a mov avg
    # coeff = 1/N, N being the window size
    # modes are valid, full and same
    moving_average = np.convolve(y_n, np.ones((N_1,)) / N_1, mode=mode)
    # repeat for beat region of interest and beat mov avg
    return moving_average

### Skin Temperature Window & Main Code

In [ ]:
def temperaturewindowing(df,windowsizerow,state):
  feature=['Mean_Temp','STD_Temp','Min_Temp','Max_Temp','Min_MovAvg_Temp','Max_MovAvg_Temp','Mean_MovAvg_Temp','STD_MovAvg_Temp','stress']
  result_df= pd.DataFrame(columns=feature) 
  df1=df.rolling(window=windowsizerow)
  type(df1)
  i=1
  j=0
  for df2 in df1:
    if len(df2)>=windowsizerow and i%int(windowsizerow/2)==0:
      
      results={}
      mov_avg = _generate_moving_averages(df2.values.flatten(), sampling_frequency=64, window=windowsizerow/64,mode='same')  
      results['Min_MovAvg_Temp'] = np.min(mov_avg)
      results['Max_MovAvg_Temp'] = np.max(mov_avg)
      results['Mean_MovAvg_Temp'] = statistics.mean(mov_avg)
      results['STD_MovAvg_Temp'] = statistics.stdev(mov_avg)                                                 
      results['Mean_Temp'] = np.mean(df2.values.flatten())
      results['STD_Temp'] = np.std(df2.values.flatten())
      results['Min_Temp'] = np.min(df2.values.flatten())
      results['Max_Temp'] = np.max(df2.values.flatten())
      results['stress'] = state
      
      result_df=result_df.append(results,ignore_index=True)
   
    i+=1
    j+=1
    
  return result_df

In [ ]:
def SkinTemperature_FeatureExtraction(drive, index):
  annot=pd.read_csv(r'/gdrive/MyDrive/SamsungWork/AffectiveROAD_Dataset/Database/E4/Annot_E4_Left.csv')
  temp=pd.read_csv(os.path.join(r'/gdrive/MyDrive/SamsungWork/AffectiveROAD_Dataset/Database/ExtractedE4Files/E4',drive,"Left","TEMP.csv"),names=['TEMP'])
  temperature=temp[2:]
  temperature.reset_index(drop=True,inplace=True)
  mul_factor=64//4
  
  upsampled_temperature=signal.resample(temperature,len(temperature)*mul_factor)
  temperature = pd.DataFrame(upsampled_temperature, columns =['TEMP'])

  temperaturefeaturesname=['Mean_Temp','STD_Temp','Min_Temp','Max_Temp','Min_MovAvg_Temp','Max_MovAvg_Temp','Mean_MovAvg_Temp','STD_MovAvg_Temp','stress']

  temperatureresultdf= pd.DataFrame(columns=temperaturefeaturesname) 



  # rest features computation 
  rest=temperature[annot['Rest_Start'][index]*mul_factor:annot['Rest_End'][index]*mul_factor]
  rest=rest.append(temperature[annot['Rest_Start.1'][index]*mul_factor:annot['Rest_End.1'][index]*mul_factor],ignore_index=True)
  rest.reset_index(drop=True,inplace=True)
  restfeatures=temperaturewindowing(rest,64*10,0)
  temperatureresultdf=temperatureresultdf.append(restfeatures,ignore_index=True)

    # medium stress features computation-> Highway
  medium=temperature[annot['Hwy_Start'][index]*mul_factor:annot['Hwy_End'][index]*mul_factor]
  medium=medium.append(temperature[annot['Hwy_Start.1'][index]*mul_factor:annot['Hwy_End.1'][index]*mul_factor],ignore_index=True)
  medium.reset_index(drop=True,inplace=True)
  mediumfeatures=temperaturewindowing(medium,64*10,1)
  temperatureresultdf=temperatureresultdf.append(mediumfeatures,ignore_index=True)

    # high stress features computation-> cities
  high=temperature[annot['City1_Start'][index]*mul_factor:annot['City1_End'][index]*mul_factor]
  high=high.append(temperature[annot['City1_Start.1'][index]*mul_factor:annot['City1_End.1'][index]*mul_factor],ignore_index=True)
  high=temperature[annot['City2_Start'][index]*mul_factor:annot['City2_End'][index]*mul_factor]
  high=high.append(temperature[annot['City2_Start.1'][index]*mul_factor:annot['City2_End.1'][index]*mul_factor],ignore_index=True)
  high.reset_index(drop=True,inplace=True)
  highfeatures=temperaturewindowing(high,64*10,2)
  temperatureresultdf=temperatureresultdf.append(highfeatures,ignore_index=True)

  return temperatureresultdf



# Feature Extraction on Files

In [ ]:
outpath_path=r'/gdrive/MyDrive/SamsungWork/AffectiveROADWork/27MayFeatures'

In [ ]:
def features(drive, index):
  BVPFeatures=BVP_FeatureExtraction(drive, index)
  GSRFeatures=GSR_FeatureExtraction(drive, index)
  TemperatureFeatures=SkinTemperature_FeatureExtraction(drive, index)

  # saving to files
    # setting up folders for saving
  if not os.path.exists(os.path.join(outpath_path,drive)):
    os.makedirs(os.path.join(outpath_path,drive))

  BVPFeatures.to_csv(os.path.join(outpath_path,drive,"BVPFeatures.csv"))
  GSRFeatures.to_csv(os.path.join(outpath_path,drive,"GSRFeatures.csv"))
  TemperatureFeatures.to_csv(os.path.join(outpath_path,drive,"SkinTemperatureFeatures.csv"))



In [ ]:
files=['1-E4-Drv1', 
 '2-E4-Drv2',
 '3-E4-Drv3',
 '4-E4-Drv4',
 '5-E4-Drv5',
 '6-E4-Drv6',
 '7-E4-Drv7', 
 '8-E4-Drv8', 
 '9-E4-Drv9',
 '10-E4-Drv10',
  '11-E4-Drv11',
 '12-E4-Drv12',
 '13-E4-Drv13']

In [ ]:
for i, val in enumerate(files):
    features(val,i)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 105, using nperseg = 105
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 131, using nperseg = 131
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 141, using nperseg = 141
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 134, using nperseg = 134
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 152, using nperseg = 152
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 130, using nperseg = 130
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 148, using nperseg = 148
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 137, using nperseg = 137
  .format(nperseg, input_length))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


...

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 83, using nperseg = 83
  .format(nperseg, input_length))


Streaming output truncated to the last 5000 lines.
 6: -1.6398e+03 -1.6398e+03  2e-03  4e-08  3e-08
 7: -1.6398e+03 -1.6398e+03  3e-04  6e-09  5e-09
 8: -1.6398e+03 -1.6398e+03  2e-04  2e-09  1e-09
 9: -1.6398e+03 -1.6398e+03  5e-05  4e-10  3e-10
10: -1.6398e+03 -1.6398e+03  2e-05  1e-10  8e-11
11: -1.6398e+03 -1.6398e+03  8e-06  3e-11  2e-11
12: -1.6398e+03 -1.6398e+03  2e-06  2e-12  3e-12
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.5786e+03 -1.5779e+03  6e+02  3e+01  2e+01
 1: -1.5777e+03 -1.5955e+03  2e+01  7e-01  5e-01
 2: -1.5781e+03 -1.5799e+03  2e+00  5e-02  4e-02
 3: -1.5784e+03 -1.5787e+03  3e-01  5e-04  4e-04
 4: -1.5786e+03 -1.5786e+03  4e-02  6e-05  5e-05
 5: -1.5786e+03 -1.5786e+03  9e-03  1e-05  8e-06
 6: -1.5786e+03 -1.5786e+03  2e-03  2e-06  1e-06
 7: -1.5786e+03 -1.5786e+03  1e-03  7e-07  5e-07
 8: -1.5786e+03 -1.5786e+03  3e-04  2e-07  1e-07
 9: -1.5786e+03 -1.5786e+03  3e-04  9e-08  7e-08
10: -1.5786e+03 -1.5786e+03  5e-05  2e-08  1

/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 157, using nperseg = 157
  .format(nperseg, input_length))


Streaming output truncated to the last 5000 lines.
 3: -9.5360e+03 -9.5364e+03  4e-01  7e-04  2e-04
 4: -9.5363e+03 -9.5364e+03  7e-02  1e-04  3e-05
 5: -9.5364e+03 -9.5364e+03  4e-02  5e-05  2e-05
 6: -9.5364e+03 -9.5364e+03  1e-02  7e-06  2e-06
 7: -9.5364e+03 -9.5364e+03  7e-03  3e-06  8e-07
 8: -9.5364e+03 -9.5364e+03  1e-03  4e-07  1e-07
 9: -9.5364e+03 -9.5364e+03  7e-04  1e-07  3e-08
10: -9.5364e+03 -9.5364e+03  2e-04  3e-08  1e-08
11: -9.5364e+03 -9.5364e+03  7e-05  3e-10  1e-10
12: -9.5364e+03 -9.5364e+03  1e-05  5e-11  1e-11
13: -9.5364e+03 -9.5364e+03  2e-06  2e-12  1e-12
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.3525e+03 -9.3520e+03  7e+02  3e+01  8e+00
 1: -9.3521e+03 -9.3705e+03  2e+01  7e-01  2e-01
 2: -9.3527e+03 -9.3577e+03  5e+00  2e-01  5e-02
 3: -9.3529e+03 -9.3554e+03  2e+00  5e-02  2e-02
 4: -9.3530e+03 -9.3539e+03  9e-01  9e-03  3e-03
 5: -9.3532e+03 -9.3536e+03  4e-01  3e-03  9e-04
 6: -9.3533e+03 -9.3535e+03  2e-01  1e-03  4

# Combining Features

In [ ]:
def combine(outpath_path):
    
  EXT = "BVPFeatures.csv"
  all_csv_files = [file
                  for path, subdir, files in os.walk(outpath_path)
                  for file in glob(os.path.join(path, EXT))]
  # len(all_csv_files)
  column_names=['mean_nni','sdnn','sdsd','rmssd','median_nni','nni_50','pnni_50','nni_20','pnni_20','range_nni','cvsd','cvnni','mean_hr','max_hr','min_hr','std_hr','triangular_index','tinn','total_power','vlf','lf','hf','lf_hf_ratio','lfnu','hfnu','csi','cvi','Modified_csi','sd1','sd2','ratio_sd2_sd1','sampen','stress']

  combineddf= pd.DataFrame(columns=column_names) 

  for featuredata in all_csv_files:
      tempdata=pd.read_csv(featuredata)
      combineddf=combineddf.append(tempdata,ignore_index=True)

  combineddf.to_csv(r'/gdrive/MyDrive/SamsungWork/AffectiveROADWork/27MayFeatures/BVPCombinedFeatures.csv')

  gsrext = "GSRFeatures.csv"
  gsrfilescsv = [file
                  for gsrpath, subdir, files in os.walk(outpath_path)
                  for file in glob(os.path.join(gsrpath, gsrext))]
  # len(gsrfilescsv)
  gsrfeature=['mean_gsr','var_gsr','skew_gsr','kurtosis_gsr','std_gsr','mean_scl','var_scl','skew_scl','kurtosis_scl','std_scl','slope_scl','mean_scr','var_scr','skew_scr','kurtosis_scr','std_scr','max_scr','scr_peaks','stress']

  commbinegsr= pd.DataFrame(columns=gsrfeature) 

  for featuredata in gsrfilescsv:
      tempdata=pd.read_csv(featuredata)
      commbinegsr=commbinegsr.append(tempdata,ignore_index=True)

  commbinegsr.to_csv(r'/gdrive/MyDrive/SamsungWork/AffectiveROADWork/27MayFeatures/GSRCombinedFeatures.csv')


  skintempext = "SkinTemperatureFeatures.csv"
  gsrfilescsv = [file
                  for gsrpath, subdir, files in os.walk(outpath_path)
                  for file in glob(os.path.join(gsrpath, skintempext))]
  # len(gsrfilescsv)
  temperaturefeaturesname=['Mean_Temp','STD_Temp','Min_Temp','Max_Temp','Min_MovAvg_Temp','Max_MovAvg_Temp','Mean_MovAvg_Temp','STD_MovAvg_Temp','stress']

  combineskintemp= pd.DataFrame(columns=temperaturefeaturesname) 

  for featuredata in gsrfilescsv:
      tempdata=pd.read_csv(featuredata)
      combineskintemp=combineskintemp.append(tempdata,ignore_index=True)

  combineskintemp.to_csv(r'/gdrive/MyDrive/SamsungWork/AffectiveROADWork/27MayFeatures/SkinTemperatureCombinedFeatures.csv')





In [ ]:
combine(outpath_path)